In [1]:
from pyntcloud import PyntCloud
from pyntcloud.io import bin as io_bin
import numpy as np
import pandas as pd
from pythreejs import *  
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Text, Output, VBox
from traitlets import link, dlink
import glob
from check import ROI

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
def createBBox(bounding_box,C1,C2,C3,C4,C5,C6,C7,C8,color="yellow"):
    bounding_box.append({"color":color, "vertices":[C1,C2,C3,C4,C1]})
    bounding_box.append({"color":color, "vertices":[C1,C4,C8,C5,C1]})
    bounding_box.append({"color":color, "vertices":[C1,C2,C6,C5,C1]})
    bounding_box.append({"color":color, "vertices":[C2,C6,C7,C3,C2]})
    bounding_box.append({"color":color, "vertices":[C3,C7,C8,C4,C3]})
    return bounding_box

with open('lidar_config.txt') as f:
    lidar_config = json.load(f)
    index = len(lidar_config['lidar'])
    
home = [0,0,2.8]
CLOUD = pd.DataFrame(columns=['x','y','z'])
individual_points = []

lidar_num = index;
for i in range(lidar_num):
    pointfiles = glob.glob("convert_image/{}/Pointclouds/*".format(i))     # pointcloud files
    coord = lidar_config['lidar'][i]['location']

    num = 1
    labelfiles = glob.glob("convert_image/{}/Labels/*".format(i))
    curr_boxes = np.load(labelfiles[num])                   # bounding box vertex location files
    if(i==0):
        boxes_inside = []
        for box in curr_boxes:
            if(box[0,0] < ROI/2 and box[0,1] < ROI/2):
                print([box[0,0],box[0,1],ROI/2])
                boxes_inside.append(box)
        curr_boxes = boxes_inside

        boxes_gt = list()
        for each in curr_boxes:
            each = each.tolist()
            boxes_gt = createBBox(boxes_gt,each[0],each[1],each[2],each[3],each[4],each[5],each[6],each[7])

    points_numpy = np.loadtxt(pointfiles[num])         # specify pointcloud file
    new = np.random.random((len(points_numpy),3))
    points_numpy = np.hstack((points_numpy,new))
    points = pd.DataFrame(data=points_numpy,columns=['x', 'y', 'z','red','green','blue'])
    points.iloc[:,0:3] = points.iloc[:,0:3] - (np.asarray(coord) - np.asarray(home))
    individual_points.append(len(points))
    CLOUD = CLOUD.append(points)

cloud = PyntCloud(CLOUD)
colors = [np.array([0,0,255], dtype=np.uint8),np.array([0,255,0],dtype=np.uint8)]
for i in range(1,lidar_num+1):
    for j in range(individual_points[i-1]):
        if(i==1):
            cloud.points.loc[j,0:3] = colors[i-1]
        else:
            cloud.points.iloc[individual_points[i-2]+j-2,0:3] = colors[i-1]

[-28.84548213854492, -100.68116193737977, 25.0]
[-158.0663200397419, -14.075318990896848, 25.0]
[-29.75545013838793, -75.35218177396989, 25.0]
[-27.18468318190382, -147.20274599295064, 25.0]
[-157.5864972691074, -0.1694710734353886, 25.0]
[-59.25724383958205, -79.42368372904384, 25.0]
[-16.920711643667616, -1.1763867539796085, 25.0]
[-15.589534145194847, -91.12906675538535, 25.0]
[-130.68348541633935, 5.045987069669291, 25.0]
[-123.0356258118031, 5.995485474742516, 25.0]
[-81.09285617539317, -153.21624014991525, 25.0]
[-47.51853237615625, -55.19235131277628, 25.0]
[-58.21425158474368, -147.02574774875163, 25.0]
[-154.43378385197832, 17.530649259851714, 25.0]
[-128.37467746431463, -3.657999856654186, 25.0]
[-15.806863219791424, 12.257806589682328, 25.0]
[-126.9813687049137, -7.869570015434041, 25.0]
[-2.679646585691515, -4.726257707950888, 25.0]
[-154.81170626007017, -1.4005237453378867, 25.0]
[-95.24170439656015, 6.709435361428911, 25.0]
[7.511118849280152, -75.37990949430521, 25.0]
[-

In [5]:
cloud.plot(initial_point_size=0.02, polylines=boxes_gt)
# lidar_point_cloud = PyntCloud.from_file("convert_image/lidar/{}.ply".format(num))
# lidar_point_cloud.points = -5*lidar_point_cloud.points
# lidar_point_cloud.plot(initial_point_size=0.02)

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-1.5777414111396109, 25.13055287961429, 11.8…

In [42]:
import csv
import json
import pandas as pd

data = {}
data['lidar'] = []

# data['lidar'].append({
#     'location': [0,0,2.8,0,0,0],
#     'angles': [-25,-1,-1.667,-15.639,-11.31,0,-0.667,-8.843,-7.254,0.333,-0.333,-6.148,-5.333,
#             1.333,0.667,-4,-4.667,1.667,1,-3.667,-3.333,3.333,2.333,-2.667 ,-3,7,4.667,-2.333 ,-2,15,10.333,-1.333],
#     'resolution': 0.2
# })
data['lidar'].append({
    'location': [0,-1,2.8],
    'angles': [-15,-13,-11,-9,-7,-5,-3,-1,1,3,5,7,9,11,13,15],
    'resolution': 0.4
})
data['lidar'].append({
    'location': [0,1,2.8],
    'angles': [-25,-1,-1.667,-15.639,-11.31,0,-0.667,-8.843,-7.254,0.333,-0.333,-6.148,-5.333,
            1.333,0.667,-4,-4.667,1.667,1,-3.667,-3.333,3.333,2.333,-2.667 ,-3,7,4.667,-2.333 ,-2,15,10.333,-1.333],
    'resolution': 0.4
})

with open('lidar_config.txt', 'w') as outfile:
    json.dump(data, outfile)

In [25]:
np.shape(curr_boxes)

(151, 8, 3)